# **Ontario grid data dashboard**

Online dashboard available at https://ryanfobel.github.io/ontario-grid-data/

## Roadmap:

### **v0.1**
* [x] Automate build with github action (`panel convert notebooks/index.ipynb --to pyodide-worker --out docs --pwa --title "Ontario grid data"`)
* [x] [Add icon](https://discourse.holoviz.org/t/how-to-change-favicon/4016/4)

### **v0.2**
* [ ] [auto-refresh data](https://github.com/ryanfobel/ontario-grid-data/issues/44) (e.g., every 15min or with a button) without reloading app
* [ ] [add git versioning](https://setuptools-git-versioning.readthedocs.io/en/stable/install.html)
* [ ] Add version number, about, link to github in footer
* [ ] Add logging/debug info
* [ ] Fix apple icon and link

### **v0.3**
* [ ] [Compress/Partition](https://stackoverflow.com/questions/73551783/how-to-read-filtered-partitioned-parquet-files-efficiently-using-pandass-read-p)/cache data for memory/[profile](https://discourse.holoviz.org/t/loading-rendering-time-in-browser/2843/2) for startup time efficiency
* [ ] Refactor to make dashboard/plots configurable via yaml config files

### **v0.4**
* [ ] Download data as zipped csv
* [ ] Add electricity and gas pricing data
* [ ] Add local weather data
* [ ] Update github readme with link to dashboard

### **v0.5**
* [ ] Add tests
* [ ] Load greenbutton gas and electricity data
* [ ] backup/restore/share user data (with optional encryption keys) to local cache, google drive
* [ ] Private devices
* [ ] Fix resizing behaviour when datetime range changes (also when sidebar closes)
I'm currently using a hidden dummy plot-hack to get resizing to work when the user changes the date range. [This](https://github.com/holoviz/panel/issues/1245) looks like maybe a better solution.

An earlier workaround was to replot each figure whenever the datetime range changes (this works, but performance suffers significantly):

```python
def on_range_changed(panels, event, plots):
    for k, v in panels.items():
        v[0] = pn.panel(plots[k], sizing_mode='stretch_both')

range_select.link(panels, callbacks={'value': ft.partial(on_range_changed, plots=plots)})
```

## **v0.6**
* [ ] Calculate co2, pricing (compare reate plans), calculate carbon tax + rebate
* [ ] Fit [caltrack models](https://www.recurve.com/how-it-works/caltrack-billing-daily-methods)

## **v0.7**
* [ ] Add forecasting

In [ ]:
# imports, function defs

#%conda install -c conda-forge panel pandas numpy holoviews hvplot pyarrow arrow matplotlib python-dotenv

import datetime as dt
import io
import requests
import functools as ft
import os
from pkg_resources import get_distribution, DistributionNotFound

import panel as pn
import pandas as pd
import numpy as np
import holoviews as hv
import hvplot.pandas

import ontario_grid_data


def get_version():
    try:
        return get_distribution("ontario_grid_data").version
    except DistributionNotFound:
        # package is not installed
        return ''


# workaround for iphone
def _read_csv(url, *args, **kwargs):
    response = requests.get(url)
    return pd.read_csv(io.StringIO(response.content.decode('utf-8')), *args, **kwargs)


# helper function for getting data
def get_series(**options):
    df = _read_csv(options["url"], index_col=0, compression=None)
    df.index = pd.to_datetime(df.index, utc=True).tz_convert(options["tz"])
    df = df.sort_index(ascending=True)
    return df[[options["column"]]].rename(columns={options["column"]: options["name"]})


# helper function for getting data
def get_dataframe(**options):
    df = _read_csv(options["url"], index_col=0, compression=None)
    df.index = pd.to_datetime(df.index, utc=True).tz_convert(options["tz"])
    df = df.sort_index(ascending=True)
    return df


def utc_to_local(utc_dt):
    return utc_dt.replace(tzinfo=dt.timezone.utc).astimezone(tz=None)


def local_to_utc(local_dt, tz):
    return local_dt.tz_localize(tz=tz).astimezone(tz=dt.timezone.utc)


def load_gridwatch_data():
    options = dict(
        url="https://raw.githubusercontent.com/ryanfobel/ontario-grid-data/main/data/clean/gridwatch.ca/hourly/summary.csv",
        tz="America/Toronto",
        column="",
    )
    return get_dataframe(**options)


def plot_generation_by_source(data):
    plot_options = dict(
        value_label="MW",
        legend="bottom",
        title="Generation by source",
        height=height,
        width=width,
        grid=True,
        stacked=True,
        ylim=(0, None),
        alpha=0.4,
        hover=False,
        fontsize=fontsize,
        xlim=default_range,
    )
    rename_columns = {x: x.replace(" (MW)", "") for x in data["gridwatch"].columns if x.endswith(" (MW)") and (x[0]==x[0].lower())}
    return data["gridwatch"][rename_columns.keys()].rename(columns=rename_columns).hvplot.area(**plot_options)


def load_co2_intensity():
    options_gridwatch = dict(
        name = "gridwatch",
        url="https://raw.githubusercontent.com/ryanfobel/ontario-grid-data/main/data/clean/gridwatch.ca/hourly/summary.csv",
        tz="America/Toronto",
        column = "CO2e Intensity (g/kWh)"
    )
    
    options_co2signal = dict(
        name="co2signal",
        url="https://raw.githubusercontent.com/ryanfobel/ontario-grid-data/main/data/clean/co2signal.com/CA-ON/hourly/output.csv",
        column = "data.carbonIntensity",
        tz="America/Toronto",
    )

    return get_series(**options_gridwatch).join(
        get_series(**options_co2signal),
        how="inner"
    )


def plot_co2_emissions_intensity(data):
    plot_options = dict(
        value_label="g/kWh",
        legend="bottom",
        title="co2 emissions intensity",
        height=height,
        width=width,
        grid=True,
        ylim=(0, None),
        fontsize=fontsize,
        xlim=default_range,
        # alpha=0.5,
        # hover=False,
    )
    return data["co2_intensity"].hvplot.line(**plot_options)


def plot_supply_demand(data):
    plot_options = dict(
        value_label="MW",
        legend="bottom",
        title="Supply and demand",
        height=height,
        width=width,
        stacked=False,    
        grid=True,
        fontsize=fontsize,
        xlim=default_range,
        # ylim=(0, None),
        # alpha=0.5,
        # hover=False,
    )
    rename_columns = {
        "Power Generated (MW)": "supply",
        "Ontario Demand (MW)": "demand",
        "Imports (MW)": "imports",
        "Exports (MW)": "exports",
        "Net Import/Exports (MW)": "net (exports-imports)",
    }
    return data["gridwatch"][rename_columns.keys()].rename(columns=rename_columns).hvplot.line(**plot_options)


def plot_generation_pct(data):
    plot_options = dict(
        value_label="%",
        legend="bottom",
        title="Relative generation by source",
        height=height,
        width=width,
        grid=True,
        stacked=True,
        ylim=(0, 100),
        alpha=0.4,
        hover=False,
        fontsize=fontsize,
        xlim=default_range,
    )
    rename_columns = {x: x.replace(" (%)", "") for x in data["gridwatch"].columns if x.endswith(" (%)")}
    return data["gridwatch"][rename_columns.keys()].rename(columns=rename_columns).hvplot.area(**plot_options)


def apply_plot_options(plots, range_select):
    return {
        k:(
            v.apply.opts(active_tools=active_tools, backend_opts=backend_opts, xlim=range_select)
            if k=='dummy'
            else
            v.apply.opts(active_tools=active_tools, backend_opts=backend_opts)
        ) for k, v in plots.items()
    }


def update_plots(data, range_select):
    return apply_plot_options({
        'generation_pct': plot_generation_pct(data),
        'co2_intensity': plot_co2_emissions_intensity(data),
        'supply_demand': plot_supply_demand(data),
        'generation': plot_generation_by_source(data),
    }, range_select)


def refresh_data():
    global now, data, plots, panels, range_select
    new_data = {
        "gridwatch": load_gridwatch_data()
    }
    if new_data["gridwatch"].index[-1] > data["gridwatch"].index[-1]:
        new_data["co2_intensity"] = load_co2_intensity()
        data = new_data

        # update range
        now = data["gridwatch"].index[-1]
        range_select.value=default_range = (utc_to_local(now-dt.timedelta(days=7)), utc_to_local(now))

        plots = update_plots(new_data, range_select)
        for k, v in panels.items():
            v[0] = pn.panel(plots[k], sizing_mode='stretch_both')
        debug.value=dt.datetime.now().isoformat()

In [ ]:
# config options

hvplot.extension("bokeh")
theme = "dark"
pn.config.theme = theme
pn.extension()

# disable linking of axes
# hv.opts.defaults(
#     hv.opts.Curve(axiswise=True, framewise=True, shared_axes=False),
#     hv.opts.Area(axiswise=True, framewise=True, shared_axes=False),
#     hv.opts.Scatter(axiswise=True, framewise=True, shared_axes=False),
#     hv.opts.Image(axiswise=True, framewise=True, shared_axes=False),
#     hv.opts.Histogram(axiswise=True, framewise=True, shared_axes=False)
# )

refresh_period_minutes = 15
active_tools = []
backend_opts={"plot.toolbar.autohide": True}
height=350
width=750
fontsize = {
    'title': 24,
    'labels': 18,
    'xticks': 14,
    'yticks': 14,
    'legend': 14,
}

data = {}
plots = {}

In [ ]:
# initialize dashboard
data["gridwatch"] = load_gridwatch_data()
now = data["gridwatch"].index[-1]
start_time = now - dt.timedelta(days=365)
default_range = (utc_to_local(now-dt.timedelta(days=7)), utc_to_local(now))
range_select = pn.widgets.DatetimeRangePicker(
    value=default_range
)

data["gridwatch"] = data["gridwatch"][data["gridwatch"].index > start_time]
plots["generation"] = plot_generation_by_source(data)
data["co2_intensity"] = load_co2_intensity()
data["co2_intensity"] = data["co2_intensity"][data["co2_intensity"].index > start_time]
plots["co2_intensity"] = plot_co2_emissions_intensity(data)
plots["supply_demand"] = plot_supply_demand(data)
plots["generation_pct"] = plot_generation_pct(data)
plots["dummy"] = plot_co2_emissions_intensity(data)

button_map = {
    "Last 12 hours": dt.timedelta(hours=12),
    "Last 24 hours": dt.timedelta(hours=24),
    "Last 2 days": dt.timedelta(days=2),
    "Last 7 days": dt.timedelta(days=7),
    "Last 30 days": dt.timedelta(days=30),
    "Last 90 days": dt.timedelta(days=90),
    "Last 6 months": dt.timedelta(days=365./2),
    "Last 1 year": dt.timedelta(days=365),
}

plots = apply_plot_options(plots, range_select)

panels = {
    k: pn.Column(pn.panel(v, sizing_mode='stretch_both'))
    for k, v in plots.items()
}

buttons = [
    pn.widgets.Button(name=name, button_type='light')
    for name in button_map.keys()
]

sidebar_footer = pn.pane.Markdown(f"""
{ get_version() }
<a href="https://github.com/ryanfobel/ontario-grid-data/" target="_blank">About</a>
""")

template = pn.template.FastGridTemplate(
    row_height=200,
    theme_toggle=False,
    prevent_collision=True,
    theme=theme,
    title="Ontario grid data",
    sidebar=[range_select, *buttons, sidebar_footer, pn.panel(plots["dummy"], visible=False)],
    collapsed_sidebar=True,
    logo="images/icon-vector.svg",
)

template.main[0:2,0:6]=panels['co2_intensity']
template.main[0:2,6:12]=panels['generation_pct']
template.main[2:4,0:6]=panels['supply_demand']
template.main[2:4,6:12]=panels['generation']


def b(event):
    global range_select
    delta = button_map[event.obj.name]
    new_dates = tuple([utc_to_local(now-delta), utc_to_local(now)])
    if new_dates != range_select.value:
        range_select.value = new_dates

for button in buttons:
    button.on_click(b)

# callback not working in pwa
# pn.state.add_periodic_callback(refresh_data, int(refresh_period_minutes*60*1000))

In [ ]:
for k, v in plots.items():
    if k != "dummy":
        display(v)

<img src="images/icon-vector.svg" alt="icon" width="180"/>

renewable energy by Nawicon from <a href="https://thenounproject.com/browse/icons/term/renewable-energy/" target="_blank" title="renewable energy Icons">Noun Project</a> (CC BY 3.0)

In [ ]:
template.servable();